# Segmenting and Clustering TORONTO Neighborhoods


## ===== PART I =====
### Transform data from table in Wikipedia page to Pandas Dataframe

In [1]:
import warnings; warnings.simplefilter('ignore')

In [7]:
!conda install -c anaconda lxml -y

In [2]:
import pandas as pd
import numpy as np
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [3]:
toronto_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
toronto_df.columns = ['PostalCode', 'Borough', 'Neighborhood']
print(toronto_df.shape)
toronto_df.head(5)

(287, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# ignore rows with Borough 'Not assigned'
df = toronto_df[ toronto_df['Borough'] != 'Not assigned' ]
print(df.shape)

(210, 3)


In [5]:
# replace any'Not assigned' Neighbourhood with the value of Borough in the same row
df['Neighborhood'] = df['Borough'].where( df['Neighborhood']=='Not assigned', df['Neighborhood'] )
print(df.shape)

(210, 3)


In [7]:
# combine neighborhoods in the same postcode into one single data entry
df = pd.DataFrame( df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join) ).reset_index()
print('The number of rows of the dataframe is:', df.shape[0])
df.head()

The number of rows of the dataframe is: 103


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## === END OF PART I ===


# :) :) :) :) :) :) :) :) :) :)

## ===== PART II =====
### get the latitude and the longitude of each neighborhood using Geocoder Python package


Note: Give up on Geocoder - it kept returning nothing but "None".
      Read the latitude and longitude information from http://cocl.us/Geospatial_Data instead.

In [10]:
# convert into dataframe from http://cocl.us/Geospatial_data
df_lat_lng = pd.read_csv('http://cocl.us/Geospatial_data')
df_lat_lng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
# Combine df and df_lat_lng into df_all
# df_all is df with latitude and logitude information
df_all = pd.merge(df, df_lat_lng, how='left', left_on='PostalCode', right_on='Postal Code')
df_all.drop(['Postal Code'], axis=1, inplace=True)
df_all.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## ===== PART III =====
## Explore and Cluster Neighborhoods in Downtown Toronto
### - Map and mark neighborhoods in Downtown Toronto
### - Explore each neighborhood and analyze the composition of venues by catagory
### - Based on the composition of venues of each neighborhood, cluster similar neighborhood together into 5 clusters


## 1. Map and mark neighborhoods in Downtown Toronto
#### Visualization the neighborhoods of Downtown Toronto

In [14]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [17]:
!conda install -c conda-forge folium=0.5.0 -y # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Let's get the geographical coordinates of Downtown Toronto.

In [18]:
# List all the boroughs in Toronto
df_all['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke', "Queen's Park"], dtype=object)

In [19]:
address = 'Downtown Toronto, Toronto, ON'

geolocator = Nominatim(user_agent="Downtown_Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


Neighborhood data in Downtown Toronto

In [20]:
df_downtown_toronto = df_all[df_all['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
print(df_downtown_toronto.shape)
df_downtown_toronto.head()

(19, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


Create Map of Downtown Toronto

In [21]:
# create map of Downtown Toronto (Borough) using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(df_downtown_toronto['Latitude'], df_downtown_toronto['Longitude'], df_downtown_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

## 2. Explore each neighborhood and analyze the composition of venues by catagory


#### Let's explore the first neighborhood in our dataframe.

In [22]:
print('First Neighborhood is:', df_downtown_toronto.loc[0, 'Neighborhood'])
neighborhood_latitude = df_downtown_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_downtown_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_downtown_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

First Neighborhood is: Rosedale
Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


#### Get the top 100 venues within a radius of 500 meters of this neighborhood.

In [23]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'AJ0F5D1NRHWGU0C5EYUOG1QVX2OQYMAN2MNKM1C0QOEBLO0Z' # your Foursquare ID
CLIENT_SECRET = 'BCZ4KYDCG0D0A244M5QJ4RF1OUHKPW25P4UJIFOODX0LO3UU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [24]:
#Creat url to get the top 100 venues that are in Rosedale within a radius of 500 meters.

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=AJ0F5D1NRHWGU0C5EYUOG1QVX2OQYMAN2MNKM1C0QOEBLO0Z&client_secret=BCZ4KYDCG0D0A244M5QJ4RF1OUHKPW25P4UJIFOODX0LO3UU&v=20180605&ll=43.6795626,-79.37752940000001&radius=500&limit=100'

In [25]:
#get results of url and define next step

import requests
from pandas.io.json import json_normalize

results = requests.get(url).json()
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues

4 venues were returned by Foursquare.


,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


#### Explore Venues in Each Neighborhood in Downtown Toronto

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
toronto_downtown_venues = getNearbyVenues(names=df_downtown_toronto['Neighborhood'],
                                   latitudes=df_downtown_toronto['Latitude'],
                                   longitudes=df_downtown_toronto['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Queen's Park


In [28]:
print(toronto_downtown_venues.shape)
toronto_downtown_venues.head()

(1299, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant


In [29]:
#Let's check how many venues were returned for each neighborhood
toronto_downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",42,42,42,42,42,42
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",81,81,81,81,81,81
Christie,17,17,17,17,17,17
Church and Wellesley,83,83,83,83,83,83
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [30]:
print('There are {} uniques categories.'.format(len(toronto_downtown_venues['Venue Category'].unique())))

There are 200 uniques categories.


In [31]:
# one hot encoding
toronto_downtown_onehot = pd.get_dummies(toronto_downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_downtown_onehot['Neighborhood'] = toronto_downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_downtown_onehot.columns[-1]] + list(toronto_downtown_onehot.columns[:-1])
toronto_downtown_onehot = toronto_downtown_onehot[fixed_columns]
print(toronto_downtown_onehot.shape)
toronto_downtown_onehot.head()

(1299, 200)


,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
toronto_downtown_grouped = toronto_downtown_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_downtown_grouped.shape)
toronto_downtown_grouped

(19, 200)


,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.00
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,...,0.000000,0.00,0.00,0.00,0.011765,0.000000,0.000000,0.011765,0.000000,0.00
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.049383,0.000000,0.037037,0.012346,0.000000,0.00
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Church and Wellesley,0.012048,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,...,0.012048,0.00,0.00,0.00,0.000000,0.000000,0.012048,0.000000,0.012048,0.00
8,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.00


In [33]:
num_top_venues = 5

for hood in toronto_downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_downtown_grouped[toronto_downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1       Steakhouse  0.04
2             Café  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
         venue  freq
0  Coffee Shop  0.09
1  Cheese Shop  0.04
2   Steakhouse  0.04
3       Bakery  0.04
4         Café  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4           Airport  0.06


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.07
1                Park  0.05
2  Italian Restaurant  0.05
3              Bakery  0.05
4                Café  0.05


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.16
1   Italian Restaurant  0.06
2  Japanese Restaurant  0.04
3            Juice Bar  0.04
4       Ice Crea

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_downtown_grouped['Neighborhood']

for ind in np.arange(toronto_downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Steakhouse,Thai Restaurant,Café,Asian Restaurant,Bakery,Restaurant,Cosmetics Shop,Burger Joint
1,Berczy Park,Coffee Shop,Beer Bar,Café,Cheese Shop,Seafood Restaurant,Farmers Market,Steakhouse,Cocktail Bar,Bakery,Fountain
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Bar,Coffee Shop,Boutique
3,"Cabbagetown, St. James Town",Coffee Shop,Italian Restaurant,Park,Bakery,Restaurant,Pub,Pizza Place,Café,Chinese Restaurant,Liquor Store
4,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Sandwich Place,Burger Joint,Café,Ice Cream Shop,Gym / Fitness Center,Department Store


## 3. Based on the composition of venues of each neighborhood, cluster similar neighborhood together into 5 clusters

In [36]:
# set number of clusters
kclusters = 5

toronto_downtown_grouped_clustering = toronto_downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 3, 2, 2, 2, 4, 2, 2, 2], dtype=int32)

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_toronto_merged = df_downtown_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,2,Coffee Shop,Italian Restaurant,Park,Bakery,Restaurant,Pub,Pizza Place,Café,Chinese Restaurant,Liquor Store
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Men's Store,Fast Food Restaurant,Pub,Café
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Mexican Restaurant,Theater,Gym / Fitness Center,French Restaurant
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Cosmetics Shop,Fast Food Restaurant,Tea Room,Lingerie Store,Italian Restaurant,Ice Cream Shop


In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [39]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, 
                            downtown_toronto_merged.columns[[2] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Harbourfront,0,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Mexican Restaurant,Theater,Gym / Fitness Center,French Restaurant
18,Queen's Park,0,Coffee Shop,Gym,Park,Yoga Studio,Music Venue,Sandwich Place,Salad Place,Portuguese Restaurant,Burger Joint,Burrito Place


In [40]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, 
                     downtown_toronto_merged.columns[[2] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rosedale,1,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner


In [41]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, 
                     downtown_toronto_merged.columns[[2] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Cabbagetown, St. James Town",2,Coffee Shop,Italian Restaurant,Park,Bakery,Restaurant,Pub,Pizza Place,Café,Chinese Restaurant,Liquor Store
2,Church and Wellesley,2,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Men's Store,Fast Food Restaurant,Pub,Café
4,"Ryerson, Garden District",2,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Cosmetics Shop,Fast Food Restaurant,Tea Room,Lingerie Store,Italian Restaurant,Ice Cream Shop
5,St. James Town,2,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Bakery,Beer Bar,Cosmetics Shop,Italian Restaurant,American Restaurant
6,Berczy Park,2,Coffee Shop,Beer Bar,Café,Cheese Shop,Seafood Restaurant,Farmers Market,Steakhouse,Cocktail Bar,Bakery,Fountain
7,Central Bay Street,2,Coffee Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Sandwich Place,Burger Joint,Café,Ice Cream Shop,Gym / Fitness Center,Department Store
8,"Adelaide, King, Richmond",2,Coffee Shop,Bar,Steakhouse,Thai Restaurant,Café,Asian Restaurant,Bakery,Restaurant,Cosmetics Shop,Burger Joint
9,"Harbourfront East, Toronto Islands, Union Station",2,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Sporting Goods Shop,Brewery,Restaurant,Fried Chicken Joint,Scenic Lookout
10,"Design Exchange, Toronto Dominion Centre",2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bar,Gastropub,Steakhouse,Seafood Restaurant,Deli / Bodega
11,"Commerce Court, Victoria Hotel",2,Coffee Shop,Café,Restaurant,Hotel,Gym,Deli / Bodega,Seafood Restaurant,Steakhouse,Italian Restaurant,American Restaurant


In [42]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, 
                                             downtown_toronto_merged.columns[[2] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",3,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Bar,Coffee Shop,Boutique


In [43]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, 
                                             downtown_toronto_merged.columns[[2] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Christie,4,Grocery Store,Café,Park,Baby Store,Gas Station,Italian Restaurant,Diner,Nightclub,Candy Store,Restaurant
